# Práctica 4: Detección de características 

## Participantes:
- Gerardo León Quintana
- Susana Suárez Mendoza

In [1]:
import cv2 as cv
import numpy as np
import random
import matplotlib.pyplot as plt
import os
from transformations.transformation_factory import TransformationsFactory

## Ejercicio 1: 

Desarrolle una aplicación que permita:

a. A través de la interfaz modificar los parámetros del detector de características SIFT.

b. Seleccionar un área de interés en una imagen de elección una imagen de naturaleza médica y una imagen telemétrica. 

c. Buscar esa área de interés (recuádrela en rojo) dentro de diferentes versiones de la imagen de partida (con cambios de traslación, escala y rotación) [NOTA: Estos cambios se pueden acometer con un editor de imágenes o con el trabajo hecho en prácticas previas]. Altere mediante la interfaz las configuraciones de parámetros para mejorar la detección. 

d. Pruebe a hacer lo mismo que en el apartado c. con diferentes grados de deformación de la imagen. [NOTA: Estos cambios se pueden acometer con un editor de imágenes o con el trabajo hecho en prácticas previas].

(OPICIONAL)
A partir del detector de características “corrija” la imagen alterada para que se alinee con la imagen fuente. 

NOTA 1: Si no encuentra el área de interés la aplicación debería notificarlo. 
NOTA 2: Puede preprocesar la imagen con alguna técnica (suavizado, contrastado,…) si lo estima necesario.

In [2]:
def transform_img(img):

    img1 = TransformationsFactory.initialize_transformation('Traslacion').apply(img)
    img2 = TransformationsFactory.initialize_transformation('Rotacion').apply(img)
    img3 = TransformationsFactory.initialize_transformation('Escalado').apply(img)

    img4, img5 = TransformationsFactory.initialize_transformation('Distorciones').apply(img)

    return img1, img2, img3, img4, img5

In [3]:
def apply_sift(sift, img_orig):
    founded_matches = []

    img1, img2, img3, img4, img5 = transform_img(img_orig)
    interest_area = cv.imread('interest_area.png', cv.IMREAD_GRAYSCALE)
    keypoints1, descriptors1 = sift.detectAndCompute(interest_area, None)

    for img in [img1, img2, img3,  img4, img5]:
        keypoints2, descriptors2 = sift.detectAndCompute(img, None)

        bf = cv.BFMatcher(cv.NORM_L2, crossCheck=True)

        matches = bf.match(descriptors1, descriptors2)
        matches = sorted(matches, key=lambda x: x.distance)
        best_matches = matches[:10]
        img_matches = cv.drawMatches(interest_area, keypoints1, img, keypoints2, best_matches, None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        founded_matches.append(cv.cvtColor(img_matches, cv.COLOR_BGR2RGB))
    
    return founded_matches

In [6]:
# img = np.zeros((1024, 1024, 3), dtype=np.uint8) + 255
img = cv.imread('Tumores/tumor_glioma/Tr-gl_0516.jpg')
img_orig = img.copy()


# Parámetros de SIFT por defecto
nfeatures = 500
nOctaveLayers = 3
contrastThreshold = 0.04
edgeThreshold = 10
sigma = 1.6

coordinates = []
drawing = False 
start_point = None

def nothing(x):
    pass

def draw_shape(event, x, y, flags, param):
    global start_point, drawing, coordinates

    color = (0, 0, 255)
    thickness = 2

    if event == cv.EVENT_LBUTTONDOWN:
        drawing = True
        start_point = (x, y)
        

    # Dibujar la figura mientras se mueve el mouse
    elif event == cv.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()
            cv.rectangle(img_copy, start_point, (x, y), color, thickness)
            cv.imshow('image', img_copy)

    # Dibujar la figura al soltar el click
    elif event == cv.EVENT_LBUTTONUP:
        drawing = False
        end_point = (x, y)
        cv.rectangle(img, start_point, end_point, color, thickness)
        coordinates.append((start_point, end_point))
        cv.imshow('image', img)
        extract_interest_area(start_point, end_point, img_orig)


def extract_interest_area(start_point, end_point, img):
    x1, y1 = start_point
    x2, y2 = end_point
    if x1 < x2 and y1 < y2:
        interest_area = img[y1:y2, x1:x2]
    else:
        interest_area = img[y2:y1, x2:x1]
    cv.imwrite('interest_area.png', interest_area)



cv.namedWindow('image')
cv.setMouseCallback('image', draw_shape)
cv.createTrackbar('Features', 'image', nfeatures, 800, nothing)
cv.createTrackbar('Layers', 'image', nOctaveLayers, 10, nothing)
cv.createTrackbar('Contrast', 'image', int(contrastThreshold*100), 100, nothing)
cv.createTrackbar('Edge', 'image', edgeThreshold, 100, nothing)
cv.createTrackbar('Sigma', 'image', int(sigma*10), 100, nothing)

cv.imshow('image', img)

while True:

    nfeatures = cv.getTrackbarPos('Features', 'image')
    nOctaveLayers = cv.getTrackbarPos('Layers', 'image')
    contrastThreshold = cv.getTrackbarPos('Contrast', 'image') / 100
    edgeThreshold = cv.getTrackbarPos('Edge', 'image')
    sigma = cv.getTrackbarPos('Sigma', 'image') / 10
    
    key = cv.waitKey(1) & 0xFF
    if key == 27:
        break

    elif key == ord('r'):
        img = img_orig.copy()
        coordinates = []
        os.remove('interest_area.png')

        cv.imshow('image', img)
    
    elif key == ord('d'):
        sift = cv.SIFT_create(nfeatures=nfeatures, nOctaveLayers=nOctaveLayers, contrastThreshold=contrastThreshold, edgeThreshold=edgeThreshold, sigma=sigma)
        if os.path.exists('interest_area.png'):
            founded_matches = apply_sift(sift, img_orig)
            cv.imshow('Traslacion', founded_matches[0])
            cv.imshow('Rotacion', founded_matches[1])
            cv.imshow('Escala', founded_matches[2])
            cv.imshow('Barril', founded_matches[3])
            cv.imshow('Cojin', founded_matches[4])
        else:
            print('No se ha seleccionado un área de interés')
            # TODO: mensaje por ventana P2
    
    elif key == 8:
        cv.destroyWindow('Traslacion')
        cv.destroyWindow('Rotacion')
        cv.destroyWindow('Escala')
        cv.destroyWindow('Barril')
        cv.destroyWindow('Cojin')
        
        img = img_orig.copy()
        coordinates = []
        os.remove('interest_area.png')

        cv.imshow('image', img)
        

cv.destroyAllWindows()

In [5]:
#hola